## Install and import packages

In [ ]:
#Packages that you may need to install
#!pip install plotly
#!pip install --upgrade threadpoolctl
#!pip install folium

In [ ]:
# Packages that we are going to use in this project
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


import sys 
import os

import matplotlib.pyplot as plt
import plotly.express as px

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf

from scipy.fftpack import fft
from scipy.signal import blackman
from scipy.signal import periodogram

from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
from sklearn.preprocessing import MinMaxScaler

from functions import *

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\df")
from manage_file import FILES, getPath

# STAGE 1: INCREASING EARTHQUAKES OVER TIME

## STEP 1: Read & view of the data

In [ ]:
#### IMPORTANT TO RUN ####
# read the excel with the earthquake data.
df_earth = pd.read_excel(getPath(FILES.input_earthquake))
#### IMPORTANT TO RUN ####
# Conserving only useful columns (columns that we are going to use later)
df_earth = df_earth[["time", "year", "month", "day", "latitude", "longitude", "mag", "depth", "Pais"]]

In [ ]:
# a preview of the 10 first rows, to understand the format and the structure (you can change the number of rows to show)
#df_earth.head(10)
# show the columns the non null elements of each column and the type of object that python reads. 
#df_earth.info()

### Data formatting 

In [ ]:
#### IMPORTANT TO RUN ####
#Clean string of the column country, contains several spaces in the names.
df_earth.Pais = df_earth.Pais.str.strip()

In [ ]:
#### IMPORTANT TO RUN ####
#Formatting time from string to datetime
df_earth['time']= pd.to_datetime(df_earth['time'])
df_earth['time'] = df_earth['time'].dt.strftime('%Y-%m-%dT%H:%M:%S')
df_earth['time']= pd.to_datetime(df_earth['time'])
df_earth.month = df_earth.month.astype(int)
df_earth.day= df_earth.day.astype(int)
#df_earth.time

In [ ]:
#### IMPORTANT TO RUN ####
# Sort value in case that the database is not in order on time.
df_earth = df_earth.sort_values(by="time")
df_earth = df_earth.reset_index()

### PLOT map

In [ ]:
#First plot on map. To visualize the data obtain
color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(df_earth, 
                        lat="latitude", 
                        lon="longitude", 
                        hover_name="index", 
                        hover_data=["index"],
                        #color="cluster_label",
                        #color_continuous_scale=color_scale,
                        #size="Listed",
                        zoom=1, 
                        height=800,
                        width=800)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## STEP 2:  __Frequency for a period of time of earthquakes considering magnitude segmentation.__

For ex:
* period_length = 10 (every 10 years compare)
* mag_seg = {magnitude < 4 |  4<= magnitude < 5 | 5 < magnitude }

In [ ]:
#### IMPORTANT TO RUN ####
# In this case we have data 1972-2023. 2023 is incomplete so we filter that year. 1972 is also incomplete.
df_earth = df_earth[(df_earth.year < 2023) & (df_earth.year > 1972)]

In [ ]:
#### IMPORTANT TO RUN ####
######## you must want to change these piece of code ########

period_length = 10 #variable to change according the period length.

#############################################################
# according to the period length select we create the labels for each row in the dataset.
start_year = df_earth.time.iloc[0].year
end_year = df_earth.time.iloc[-1].year
year_range = end_year - start_year
modulo = year_range % period_length
if modulo == 0:
    final_start = end_year - period_length
else:
    final_start = end_year - modulo
final_end = end_year+1
if period_length == 1:
    starts = np.arange(start_year, final_start+1, period_length).tolist()
    tuples = [(start, start+period_length) for start in starts]
    # We'll add the last period calculated earlier
    tuples.append(tuple([final_start+1, final_end]))
else:
    starts = np.arange(start_year, final_start, period_length).tolist()
    tuples = [(start, start+period_length) for start in starts]
    # We'll add the last period calculated earlier
    tuples.append(tuple([final_start, final_end]))
bins = pd.IntervalIndex.from_tuples(tuples, closed='left')

original_labels = list(bins.astype(str))
new_labels = ['{} - {}'.format(b.strip('[)').split(', ')[0], int(b.strip('[)').split(', ')[1])-1) for b in original_labels]
label_dict = dict(zip(original_labels, new_labels))
# Assign each row to a period
df_earth['PERIOD'] = pd.cut(df_earth['year'], bins=bins, include_lowest=True, precision=0)
df_earth['PERIOD'] = df_earth['PERIOD'].astype("str")
df_earth = df_earth.replace(label_dict)

In [ ]:
#### IMPORTANT TO RUN ####
######## you must want to change these piece of code ########

mag_seg = {0: [0,4.6], 1: [4.6,4.9], 2: [4.9,10]} #variable to change

#############################################################
# with the magnitudes segments defined we create those categories in the dataset.
df_earth['MAG_SEG'] = [0]*len(df_earth)
for key, value in mag_seg.items():
    df_earth.loc[(df_earth['mag']<value[1]) & (df_earth['mag']>=value[0]), 'MAG_SEG'] = key

In [ ]:
# To have an idea of how many earthquakes are in each category of mag_seg defined.
df_earth["MAG_SEG"].value_counts()

In [ ]:
# To have de distribution of magnitudes, in the original set the min is 3.3 and max is 9.1
df_earth = df_earth.sort_values(by="time")
df_earth["mag"].describe()

In [ ]:
# Histogram plot to see the count of earthquakes of each mag_seg for all the periods
fig = px.histogram(df_earth, x="MAG_SEG",
             color='PERIOD', barmode='group',
             height=400)
fig.show()

In [ ]:
# how many earthquakes for each period
df_earth.PERIOD.value_counts()

In [ ]:
# Plot of number of earthquakes for each period of time.
fig = px.histogram(df_earth, x="PERIOD",
             color='PERIOD', barmode='group',
             height=400)
fig.show()



In [ ]:
fig = px.histogram(df_earth.sort_values(by="month"), x="MAG_SEG", color='month', barmode='group', height=400)
fig.show()

In [ ]:
df = df_earth.copy()
df['NewDate'] = df['time'] + pd.offsets.DateOffset(days=-5)

# extract the new month label from the shifted date
df['NewMonth'] = df['NewDate'].dt.month

In [ ]:
# plot a histogram with Plotly
fig = px.histogram(df.sort_values(by="NewMonth"), x="MAG_SEG", color='NewMonth', barmode='group', height=400)
#fig.update_layout(xaxis_title='MAG_SEG', yaxis_title='Count')
fig.show()

In [ ]:
df_monthly

### Country increasing

In [ ]:
# To see number of earthquakes for each country.
df_earth.Pais = df.Pais.fillna("No_Country")
for i,v in df_earth.Pais.value_counts().items():
    print(f"{i} : {v}")
    
# Calculate the number of NAN in country column
print(f'Number of NAN : {df_earth.Pais.isna().sum()}')

In [ ]:
# Plot the countries selected  to see the number of earthquakes for each period.
######## you must want to change these piece of code ########

_country = ["Indonesia", "Japan", "Chile"]

##############################################################
df_ = df_earth[df_earth.Pais.isin(_country)]
fig = px.histogram(df_, x="Pais",
             color='PERIOD', barmode='group',
             height=400)
fig.show()

In [ ]:
#### IMPORTANT TO RUN ####
# Function to calculte the trendline.
def trendline(data, order=1):
    x_ = np.arange(0,len(data))
    coeffs = np.polyfit(x_, list(data), order)
    slope = coeffs[0]
    return float(slope)

In [ ]:
# Calculate the trendline for the whole series of data, and also by magnitude segment define above.
series_totrend = df_earth.PERIOD.value_counts()
series_totrend = series_totrend.sort_index()
print(f"Trendline for the whole series is : {trendline(series_totrend)}")
trend_results = {}
for mag_seg in df_earth.MAG_SEG.unique().tolist():
    df_filt = df_earth[df_earth.MAG_SEG == mag_seg]
    series_totrend = df_filt.PERIOD.value_counts()
    series_totrend = series_totrend.sort_index()
    trend_results[f"Mag Segment {mag_seg}"] = trendline(series_totrend)
    
for key, value in trend_results.items():
    print(f"For{key} the trendline is {value}")

## Calculate period : seasonal descompose , acf, fourier and periodogram.

In [ ]:
# Plots to descompose the time series of earthquakes to find seasonality
df_ts = df_earth['year'].value_counts()
df_ts = df_ts.sort_index()

# Plot the time series data
plt.plot(df_ts)

# Calculate and plot the autocorrelation function
plot_acf(df_ts, lags=50)

# Decompose the time series into seasonal, trend, and residual components
decomposition = seasonal_decompose(df_ts, period=12)
fig = decomposition.plot()

# Display the plots
#plt.show()

In [ ]:
# Statistical algorithm to find the period of seasonality
N = len(df_ts)
w = blackman(N)
y = df_ts * w
y = fft(y.values)
f = np.linspace(0, 1, N)

# Find the dominant frequency (maximum amplitude)
idx = np.argmax(np.abs(y))
freq = f[idx]

# Calculate the period of the seasonality
period = 1 / freq

# Plot the frequency spectrum
fig, ax = plt.subplots()
ax.plot(f, np.abs(y))
ax.set_xlabel('Frequency (cycles per time unit)')
ax.set_ylabel('Amplitude')
plt.show()

In [ ]:
# Periodogram seasonality
f, Pxx = periodogram(df_ts, fs=1)
# Plot the periodogram
fig, ax = plt.subplots()
ax.plot(f, Pxx)
ax.set_xlabel('Frequency (cycles per time unit)')
ax.set_ylabel('Power Spectral Density')
plt.show()
max_idx = np.argmax(Pxx)
max_freq = f[max_idx]

print('Maximum point: frequency = {:.4f}, PSD = {:.4f}'.format(max_freq, Pxx[max_idx]))
period = 1 / max_freq
print('Period of seasonality: {:.2f} time units'.format(period))

## STEP 3: Clustering based on longitude - latitude

In [ ]:
#### IMPORTANT TO RUN ####

#drop in case of nan, but in the original case we dont find any nan
df_earth.dropna(axis=0,how='any',subset=['latitude','longitude'],inplace=True) 


coords = df_earth[['latitude', 'longitude']]
# Normalize the coordinates using min-max normalization
scaler = MinMaxScaler()
normalized_coords = scaler.fit_transform(coords)

## Clustering (find best number of clusters)

In [ ]:
def best_number_of_clusters1(coords):
    # Specify the range of k values to test
    k_range = range(10, 70)

    # Create a list to store the SSE values for each k
    sse = []

    # Loop over each value of k and compute the SSE
    for k in k_range:
        model = KMeans(n_clusters=k)
        model.fit(coords)
        sse.append(model.inertia_)

    # Plot the SSE values against k
    plt.plot(k_range, sse)
    plt.xlabel('Number of clusters')
    plt.ylabel('SSE')
    plt.show()


In [ ]:
def best_number_of_clusters2(coords):
    K_clusters = range(10,70)
    kmeans = [KMeans(n_clusters=i) for i in K_clusters]
    Y_axis = coords[['latitude']]
    X_axis = coords[['longitude']]
    score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]
    # Visualize
    plt.plot(K_clusters, score)
    plt.xlabel('Number of Clusters')
    plt.ylabel('Score')
    plt.title('Elbow Curve')
    plt.show()

In [ ]:
# I dont recommend to run this cell, take a long time to compute. I have comment those functions.

#best_number_of_clusters1(coords)
#best_number_of_clusters2(coords)




## Clustering and plot result

In [ ]:
#### IMPORTANT TO RUN ####
# Calculte and create the clustering. HERE I USE COORDS
kmeans = KMeans(n_clusters =60 , init ='k-means++')

kmeans.fit(coords) # Compute k-means clustering.
df_earth['cluster_label'] = kmeans.fit_predict(coords)
labels = kmeans.predict(coords) # Labels of each point
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.

In [ ]:
# Plot of the dataset with cluster label on map.

color_scale = [(0, 'yellow'), (1,'blue')]

fig = px.scatter_mapbox(df_earth, 
                        lat="latitude", 
                        lon="longitude", 
                        hover_name="index", 
                        hover_data=["index"],
                        color="cluster_label",
                        color_continuous_scale=color_scale,
                        #size="Listed",
                        zoom=1, 
                        height=800,
                        width=800)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#### IMPORTANT TO RUN ####
# Calculte and create the clustering. HERE I USE NORMALIZED COORDS (corods change to a normal distribution)
#kmeans = KMeans(n_clusters =60 , init ='k-means++')

#kmeans.fit(normalized_coords) # Compute k-means clustering.
#df_earth['cluster_label'] = kmeans.fit_predict(normalized_coords)
#labels_norm = kmeans.predict(normalized_coords) # Labels of each point
#centers_norm = kmeans.cluster_centers_ # Coordinates of cluster centers.

color_scale = [(0, 'orange'), (1,'red')]
df_filt = df_earth[df_earth.cluster_label.isin([0, 13, 58, 8])]
fig = px.scatter_mapbox(df_filt, 
                        lat="latitude", 
                        lon="longitude", 
                        hover_name="index", 
                        hover_data=["index"],
                        color="cluster_label",
                        #color_continuous_scale=color_scale,
                        #size="Listed",
                        zoom=1, 
                        height=800,
                        width=800)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# Plot of the same data but in a normal graph. Not pretty useful.
df_earth.plot.scatter(x = 'longitude', y = 'latitude', c=labels, s=2, cmap='plasma')
plt.scatter(centers[:, 1], centers[:, 0], c='black', s=4, alpha=0.5)
plt.show()

In [ ]:
# Calculate trendline by cluster , FOR ALL CLUSTERS
trend_results_cluster = {}
for c in df_earth["cluster_label"].unique().tolist():
    df_filt = df_earth[df_earth.cluster_label == c]
    series_totrend = df_filt.PERIOD.value_counts()
    series_totrend = series_totrend.sort_index()
    trend_results_cluster[f"Cluster {c}"] = round(trendline(series_totrend),2)
for key, value in trend_results_cluster.items():
    print(f"For {key} the trendline is {value}")

In [ ]:
trend_results_cluster =dict(sorted(trend_results_cluster.items(), key=lambda item: item[1], reverse=True))
trend_results_cluster

In [ ]:
# Select the cluster of your interest
cluster = 8
df_filt = df_earth[df_earth.cluster_label == cluster]
print(f"Cantidad de data del cluster {cluster} es {len(df_filt)}")
series_totrend = df_filt.PERIOD.value_counts()
series_totrend = series_totrend.sort_index()
print(series_totrend)
print(f"Cluster {cluster} has a trend of {trendline(series_totrend)}")
#for key, value in trend_results_cluster.items():
#    print(f"For {key} the trendline is {value}")   

# STAGE 2:  Relation between moon and eartquakes

In [ ]:
#first read moon data
df_moon = pd.read_excel(getPath(FILES.input_moon))

#Use depth to filter. if the deppth one are connected to the moon.

In [ ]:
# Change columns name and reorder
df_moon.columns = ["year", "day", "month", "acum_day", "ill_frac", "r/km", "dec", "ra/h", "ra/°"]
# 384400km
# ill frac new and full moon
# 

In [ ]:
# Fill column year.
df_moon.year = df_moon.year.replace({"common year": np.NaN, "leap year": np.NaN})
df_moon.year.ffill(inplace=True)
df_moon.year = df_moon.year.astype(int)
df_moon.dropna(inplace=True)
df_moon.month = df_moon.month.astype(int)
df_moon.day= df_moon.day.astype(int)
df_moon.acum_day= df_moon.acum_day.astype(int)

In [ ]:
df_moon["r/km"].describe()

In [ ]:
df_moon.year.value_counts()

In [ ]:
df_moon = df_moon.sort_index()
df_moon.head(35)

In [ ]:
df_earth.info()

In [ ]:
df_moon.info()

In [ ]:
# Merged data earthquake with data moon
# df earthquake : year, month, day
# df moon       : Year, Month, date
df_merged = pd.merge(df_earth, df_moon, left_on=['year', 'month', 'day'], right_on=['year', 'month', 'day'], how="left")


In [ ]:
# Functions to perfom interpolation of a certain variable.
def interpolate_position(original_position, final_position, datetime):
    """
    Perform a linear interpolation of the position of the moon at a specific datetime.

    Parameters:
    original_position (float): The position of the moon at the beginning of the day.
    final_position (float): The position of the moon at the end of the day.
    datetime: The datetime in the format 'yyyy-mm-dd hh:mm:ss'.
    Returns:
    float: The interpolated position of the moon at the specified datetime.
    """

    # Extract year, day, and month information from the datetime object
    year, day, month = datetime.year, datetime.day, datetime.month


    # Compute the fraction of the day passed by
    time_passed = (datetime - datetime.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
    total_time = (pd.to_datetime(datetime.date()) + pd.DateOffset(days=1) - pd.to_datetime(datetime.date())).total_seconds()

    # Interpolate the position of the moon linearly based on the fraction of the day passed by
    interpolated_position = original_position + (final_position - original_position) * time_passed / total_time
    #print("datetime: ", datetime)
    #print(f"Original Position {original_position} - Final Position {final_position} - interpolated_position: {interpolated_position}")
    return interpolated_position

def apply_interpolation(df_merged, df_moon, var_name=["r/km"]):
    count = 0
    df_merged[f"{var_name}_interpolated"] = [0]*len(df_merged)
    for index, row in df_merged.iterrows():
        next_day = pd.to_datetime(row["time"] + pd.DateOffset(days=1))
        try:
            next_row = df_moon[(df_moon.year == next_day.year) & (df_moon.month == next_day.month) & (df_moon.day == next_day.day)].iloc[0]
        except:
            next_row = None
        if (not np.isnan(row[var_name])) and (not next_row is None) :
            for var_name in vars_names:
                datetime = row["time"]
                original_pos = row[var_name]
                final_pos    = next_row[var_name]
                df_merged.loc[index,f"{var_name}_interpolated"] = interpolate_position(original_pos, final_pos, datetime)
            
    return df_merged

In [ ]:
var_name = "r/km" # variable to interpolate
df_merged = apply_interpolation(df_merged, df_moon, var_name) #This line apply the interpolation. This line will take time

In [ ]:
df_merged
#10-0-10 new
#90-100-90  full

In [ ]:
df_merged[df_merged["r/km_interpolated"]==0]

In [ ]:
df_merged.to_csv("minalbe_.csv")